In [18]:
# system imports
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz


# misc
import warnings

import glob

##
import pytz
import datetime as dt
import math
import seaborn as sns

import pickle

In [5]:

#style settings
sns.set(style='whitegrid', rc={'axes.facecolor': 'white'})

#sns.set_style({'font.family':'Arial', 'font.serif':'Times New Roman'})

## Related functions

In [10]:
def get_non_nan_values_cols(p_df, col_name):
    
    slic_ind=p_df[col_name].isnull()
    list_ne=slic_ind.tolist()
    false_indices = [i for i in range(len(list_ne)) if not list_ne[i]]
    
    list_col_interest=p_df[col_name].to_list()
    
    non_values_list = []
    
    for false_ind in false_indices:
        non_val = list_col_interest[false_ind]
        non_values_list.append(non_val)
    
    return non_values_list


def get_file_names_from_ind_list(files_list, inds_list):
    
    selected_files_list=[files_list[sel_index] for sel_index in inds_list]
    
    return selected_files_list


def get_sk_hr_std_mean_acc(sk_hr_files_list, condition_files):
    #event_interst_col = "event_interst"
    #mean_sk_col = "mean_sk_cond"
    #std_sk_col ="std_sk_cond"

    #mean_hr_col = "mean_hr"
    #std_hr_col = "std_hr"

    #mean_sk_acc=[]
    #std_sk_acc = []

    #mean_hr_acc=[]
    #std_hr_acc=[]

    for selected_path in sk_hr_files_list:
        dir_path, file_name=os.path.split(selected_path)
        sub_id = file_name[0:12]
        #print(sub_id)
        for cond_file in condition_files:
            if sub_id==cond_file:
                #print(selected_path)
                
                with open(selected_path, 'rb') as f:
                    ibi_data = pickle.load(f)
                    
                #sk_hr_df = pd.read_csv(selected_path)
                
                subject_id = ibi_data["subject_id"]
                event_interst = ibi_data["event_interst"]
                
                all_rr_one_list = ibi_data["all_rr_one_list"]
                
                all_rr_sublists = ibi_data["all_rr_sublists"]
                
                
                
                

                #mean_sk = sk_hr_df[mean_sk_col].values
                std_sk = sk_hr_df[std_sk_col].values
                mean_hr =sk_hr_df[mean_hr_col].values
                std_hr =sk_hr_df[std_hr_col].values


                mean_sk_acc.append(mean_sk)
                std_sk_acc.append(std_sk)
                mean_hr_acc.append(mean_hr)
                std_hr_acc.append(std_hr)



    mean_mean_sk=np.mean(np.array(mean_sk_acc), axis=0) 
    #std_mean_sk = np.std(np.array(std_sk_acc), axis=0)
    std_mean_sk = np.mean(np.array(std_sk_acc), axis=0)

    mean_mean_hr=np.mean(np.array(mean_hr_acc), axis=0) 
    #std_mean_hr = np.std(np.array(std_hr_acc), axis=0)
    std_mean_hr = np.mean(np.array(std_hr_acc), axis=0)
    
    selected_events = sk_hr_df[event_interst_col].to_list()
    
    extracted_data={"selected_event": selected_events, "hr_mean": mean_mean_hr, "hr_std": std_mean_hr, "sk_mean": mean_mean_sk, "sk_std":std_mean_sk}
    
    return extracted_data


In [ ]:
def timedomain(rr):
    results = {}

    hr = 60000/rr
    
    results['Mean RR (ms)'] = np.mean(rr)
    results['STD RR/SDNN (ms)'] = np.std(rr)
    results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
    results['Mean HR (beats/min)'] = np.mean(hr)
    results['STD HR (beats/min)'] = np.std(hr)
    results['Min HR (beats/min)'] = np.min(hr)
    results['Max HR (beats/min)'] = np.max(hr)
    results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
    results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
    results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
    return results

In [7]:
sub_data_path = "/home/muhammad/Desktop/Datasets/data_sony_digiRelax/subjective_data/subjective_data.xlsx"
#hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp3/results_22sub/hrv_analysis"
hrv_data_dir = "/home/muhammad/Desktop/repos_ixp/tester_sony_digirelax/scripts/wp4/results_all_sub/hrv/ibi_pkl"


In [8]:
sk_hr_files_list = glob.glob(hrv_data_dir+ "/*.pkl")

In [11]:
sub_data_df=pd.read_excel(sub_data_path)
file_col = "subject"
cond_col ="condition"

file_list = get_non_nan_values_cols(sub_data_df,file_col)
cond_list = get_non_nan_values_cols(sub_data_df, cond_col)

cond_set=set(cond_list)
unique_conds_list= list(cond_set)

In [12]:
print(unique_conds_list)

['2D_scent', 'VR_scent', 'VR_no_scent', '2D_no_scent']


In [13]:
no_scent_2d_inds=np.where(np.array(cond_list)=="2D_no_scent")
no_scent_2d_inds = list(no_scent_2d_inds[0])
no_scent_2d_files =get_file_names_from_ind_list(file_list, no_scent_2d_inds)

scent_2d_inds=np.where(np.array(cond_list)=="2D_scent")
scent_2d_inds=list(scent_2d_inds[0])
scent_2d_files=get_file_names_from_ind_list(file_list, scent_2d_inds)

no_scent_vr_inds = np.where(np.array(cond_list)=="VR_no_scent")
no_scent_vr_inds=list(no_scent_vr_inds[0])
no_scent_vr_files=get_file_names_from_ind_list(file_list, no_scent_vr_inds)

scent_vr_inds =np.where(np.array(cond_list)=="VR_scent")
scent_vr_inds =list(scent_vr_inds[0])
scent_vr_files=get_file_names_from_ind_list(file_list, scent_vr_inds)

In [14]:
scent_vr_files[0]

'VP006_091123'

In [19]:
pkl_file = sk_hr_files_list[0]

In [21]:
with open(pkl_files, 'rb') as f:
    ibi_data = pickle.load(f)


In [24]:
ibi_data.keys()

dict_keys(['subject_id', 'event_interst', 'all_rr_one_list', 'all_rr_sublists'])